## Problem Statement

You will have to use PINNs to solve the fluid flow for the given geometry and flow parameters.

A 2D chip is placed inside a 2D channel. The flow enters inlet at $u=1.5\text{ m/s}$ and exits through the outlet which is a $0 Pa$. All the other walls are treated as no-slip. The kinematic viscosity $(\nu)$ for the flow is $0.02 \text{ }m^2/s$ and the density $(\rho)$ is $1 \text{ }kg/m^3$. The problem is shown in the figure below.

<img src="chip_2d_geom.png" alt="Drawing" style="width: 800px;"/>

## Challenge

The main challenge in this problem is to correctly formulate the problem using PINNs. In order to achieve that, you will have to complete the following parts successfully:
1. Define the correct geometry for the problem
2. Set-up the correct boundary conditions and equations
3. Create the neural network and solve the problem

A successful completion of the problem should result in distribution of flow variables as shown below. Also, you should aim to achieve a relative $L_2$ error of less than 0.2 for all the variables w.r.t the given OpenFOAM solution. 

<img src="challenge_results.png" alt="Drawing" style="width: 650px;"/>

In this template, we will have give you a skeleton code which you will fill in to define and solve the problem.

**Note: You need to edit the `chip_2d_template.py` script that is placed in the ../source_code/chip_2d/ directory.**

From the top menu, click on File, and Open `chip_2d_template.py` from the current directory at `../source_code/chip_2d` directory. Remember to SAVE your code after changes, before running below cells.


Let us start with importing the required packages. Pay attention to the various modules and packages that are being imported, especially the equations, geometry and architectures. 

```python
from sympy import Symbol
import numpy as np
import tensorflow as tf
from simnet.solver import Solver
from simnet.dataset import TrainDomain, ValidationDomain
from simnet.data import Validation
from simnet.sympy_utils.geometry_2d import Rectangle, Line, Channel2D
from simnet.sympy_utils.functions import parabola
from simnet.csv_utils.csv_rw import csv_to_dict
from simnet.PDES.navier_stokes import IntegralContinuity, NavierStokes
from simnet.controller import SimNetController
from simnet.architecture import FourierNetArch
```

Now, define the simulation parameters and generate the geometry. You will be using the 2D geometry modules for this example. Please fill in the appropriate values for each geometry. Remember, `Channel2D` and `Rectangle` are defined by its two endpoints. The difference between a channel and rectangle in SimNet is that, the channel geometry does not have bounding curves in the x-direction. This is helpful in getting a signed distance field that it infinite in x-direction. This can be important when the signed distance field is used as a wall distance in some of the turbulence models (Refer *SimNet User Guide Chapter 3* for more details). Hence we will create the inlet and outlet using `Line` geometry (*Please note that this is a 2d line, as opposed to the* `Line1D` *that was used in the diffusion tutorial*)

```python
# simulation params
channel_length = (-2.5, 2.5)
channel_width = (-0.5, 0.5)
chip_pos = -1.0
chip_height = 0.6
chip_width = 1.0
inlet_vel = 1.5
```

```python
#TODO: Replace x1, y1, x2, y2, and X's with appropriate values

# define geometry
# define channel
channel = Channel2D((x1, y1), (x2, y2))
# define inlet and outlet
inlet = Line((x1, y1), (x1, y2), normal= X)
outlet = Line((x1, y1), (x1, y2), normal= X)
# define the chip
rec = Rectangle((x1, y1), (x2, y2))
# create a geometry for higher sampling of point cloud near the fin
flow_rec = Rectangle((chip_pos-0.25, channel_width[0]),
                     (chip_pos+chip_width+0.25, channel_width[1]))
# fluid area
geo = channel - rec
geo_hr = geo & flow_rec
geo_lr = geo - flow_rec
```

The current problem is a channel flow with an incompressible fluid. In such cases, the mass flow rate through each cross-section of the channel and in turn the volumetric flow is constant. This can be used as an additional constraint in the problem which we will help us in improving the speed of convergence. 

Wherever, possible, using such additional knowledge about the problem can help in better and faster solutions. More examples of this can be found in the *SimNet User Guide Chapter 6*.   

```python
# Optional integral continuity planes to speed up convergence 
x_pos = Symbol('x_pos')
integral_line = Line((x_pos, channel_width[0]),
                     (x_pos, channel_width[1]),
                     1)
x_pos_range = {x_pos: lambda batch_size: np.full((batch_size, 1), np.random.uniform(channel_length[0], channel_length[1]))}
```

Now you will use the created geometry to define the training data for the problem. Implement the required boundary conditions and equations in the `Chip2DTrain` class below. Remember that you will have to create the training data points both for the boundary condition and to reduce the equation residuals. You can refer to the `NavierStokes` class from the PDEs module to check how the equations are defined and the keys required for each equation. For ease of access, we show the equations below. 

```python
# Equation definitions of the NavierStokes class

# Note, this block is only for reference. Do not include this in your final script
# These equations are already present in the NavierStokes class definition

    # set equations
    self.equations = Variables()
    self.equations['continuity'] = rho.diff(t) + (rho*u).diff(x) + (rho*v).diff(y) + (rho*w).diff(z)
    self.equations['momentum_x'] = ((rho*u).diff(t)
                                  + (u*((rho*u).diff(x)) + v*((rho*u).diff(y)) + w*((rho*u).diff(z)) + rho*u*(curl))
                                  + p.diff(x)
                                  - (-2/3*mu*(curl)).diff(x)
                                  - (mu*u.diff(x)).diff(x)
                                  - (mu*u.diff(y)).diff(y)
                                  - (mu*u.diff(z)).diff(z)
                                  - (mu*(curl).diff(x)))
    self.equations['momentum_y'] = ((rho*v).diff(t)
                                  + (u*((rho*v).diff(x)) + v*((rho*v).diff(y)) + w*((rho*v).diff(z)) + rho*v*(curl))
                                  + p.diff(y)
                                  - (-2/3*mu*(curl)).diff(y)
                                  - (mu*v.diff(x)).diff(x)
                                  - (mu*v.diff(y)).diff(y)
                                  - (mu*v.diff(z)).diff(z)
                                  - (mu*(curl).diff(y)))
    self.equations['momentum_z'] = ((rho*w).diff(t)
                                  + (u*((rho*w).diff(x)) + v*((rho*w).diff(y)) + w*((rho*w).diff(z)) + rho*w*(curl))
                                  + p.diff(z)
                                  - (-2/3*mu*(curl)).diff(z)
                                  - (mu*w.diff(x)).diff(x)
                                  - (mu*w.diff(y)).diff(y)
                                  - (mu*w.diff(z)).diff(z)
                                  - (mu*(curl).diff(z)))

    if self.dim == 2:
      self.equations.pop('momentum_z')
```

Now use this understanding to define the problem. An example of the inlet boundary condition is shown. Also, the integral continuity constraint is already coded up for you. 

```python
#TODO: Replace all the placeholders with appropriate values

# define sympy variables to parametrize domain curves
x, y = Symbol('x'), Symbol('y')

class Chip2DTrain(TrainDomain):
  def __init__(self, **config):
    super(Chip2DTrain, self).__init__()

    # inlet
    inlet_parabola = parabola(y, channel_width[0], channel_width[1], inlet_vel)
    inlet_bc = inlet.boundary_bc(outvar_sympy={'u': inlet_parabola, 'v': 0},
                                 batch_size_per_area=64)
    self.add(inlet_bc, name="Inlet")

    # outlet
    outlet_bc = outlet.boundary_bc(outvar_sympy={placeholder},
                                   batch_size_per_area=placeholder)
    self.add(outlet_bc, name="Outlet")

    # noslip
    noslip = geo.boundary_bc(outvar_sympy={placeholder},
                             batch_size_per_area=placeholder)
    self.add(noslip, name="ChipNS")

    # interior lr
    interior_lr = geo_lr.interior_bc(outvar_sympy={placeholder},
                                     bounds={placeholder},
                                     lambda_sympy={placeholder},
                                     batch_size_per_area=placeholder)
    self.add(interior_lr, name="InteriorLR")

    # interior hr
    interior_hr = geo_hr.interior_bc(outvar_sympy=placeholder,
                                     bounds=placeholder,
                                     lambda_sympy=placeholder,
                                     batch_size_per_area=placeholder)
    self.add(interior_hr, name="InteriorHR")


    # integral continuity
    for i in range(4):
      IC = integral_line.boundary_bc(outvar_sympy={'integral_continuity': 1.0},
                                     batch_size_per_area=512,
                                     lambda_sympy={'lambda_integral_continuity': 1.0},
                                     criteria=geo.sdf>0,
                                     param_ranges=x_pos_range,
                                     fixed_var=False)
      self.add(IC, name="IntegralContinuity_"+str(i))
```

Now, add validation data to the problem. The examples folder contains a `openfoam` directory that contains the solution of same problem using OpenFOAM solver. The CSV file is read in and converted to a dictionary for you. Now, you will have to complete the definition of `Chip2DVal` class below. 

```python
#TODO: Set the appropriate normalization for the validation data
# The validation data has domain extents of (0,0) to (5,1). Normalize this based on your definition of the domain

# validation data
mapping = {'Points:0': 'x', 'Points:1': 'y',
           'U:0': 'u', 'U:1': 'v', 'p': 'p'}
openfoam_var = csv_to_dict('openfoam/2D_chip_fluid0.csv', mapping)
openfoam_var['x'] -= 2.5 #TODO Samle normalization of position. Edit based on your geometry definition
openfoam_var['y'] -= 0.5
openfoam_invar_numpy = {key: value for key, value in openfoam_var.items() if key in ['x', 'y']}
openfoam_outvar_numpy = {key: value for key, value in openfoam_var.items() if key in ['u', 'v', 'p']}

class Chip2DVal(ValidationDomain):
  def __init__(self, **config):
    super(Chip2DVal, self).__init__()
    val = Validation.from_numpy(placeholder)
    self.add(val, name='Val')
```

Now, complete the last part of the code by creating the `ChipSolver` to solve the problem. You will make use of an advanced architecture called the `FourierNetArch` in this problem. This architecture will help to reach the convergence faster. More details about this architecture can be found in *SimNet User Guide Section 1.6.1.1*. The important parameters of the neural network are defined for you. Feel free to tweak them and observe its behavior on the results and speed of convergence. 

```python
#TODO: Replace all the placeholders with appropriate values
class ChipSolver(Solver):
  train_domain = placeholder
  val_domain = placeholder
  arch = FourierNetArch

  def __init__(self, **config):
    super(ChipSolver, self).__init__(**config)

    self.frequencies = ('axis,diagonal', [i/5. for i in range(25)]) 
    
    self.equations = (placeholder)
    flow_net = self.arch.make_node(name='flow_net',
                                   inputs=[placeholder],
                                   outputs=[placeholder])
    self.nets = [flow_net]

  @classmethod
  def update_defaults(cls, defaults):
    defaults.update({
        'network_dir': './network_checkpoint_chip_2d',
        'rec_results': True,
        'rec_results_freq': 5000,
        'max_steps': 10000,
        'decay_steps': 100,
        'xla': True
        })
if __name__ == '__main__':
  ctr = SimNetController(ChipSolver)
  ctr.run()
```

Once you have completed all the sections of the code, you can export the notebook as a python script and then execute it as we have seen in the earlier tutorials. Record your relative L2 errors with respect to the OpenFOAM solution and try to achieve errors for all the variables lower than 0.2. Also try to visualize your results using contour plots by reading in the `.npz` files created in the network checkpoint. 



# Licensing
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0)